<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [25]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

In [2]:
# get the text

import requests

url = 'https://www.gutenberg.org/files/100/100-0.txt'
response = requests.get(url)
bill = response.text
bill[:1000]

'ï»¿\r\nProject Gutenbergâ\x80\x99s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever.  You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org.  If you are not located in the United States, youâ\x80\x99ll\r\nhave to check the laws of the country where you are located before using\r\nthis ebook.\r\n\r\n\r\nTitle: The Complete Works of William Shakespeare\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease Date: January 1994 [EBook #100]\r\nLast Updated: November 7, 2019\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***\r\n\r\n\r\n\r\n\r\nThe Complete Works of William Shakespeare\r\n\r\n\r\n\r\nby William

In [4]:
# Clean the text from numbers and symbols 
# (I'm willing to bet that Shakespeare didn't write numbers in his plays using Arabic numerals)

import re

# removes arabic numerals and other symbols
bill = re.sub(r'[^a-zA-Z]', ' ', bill)
# removes excess whitespace in the document
bill = re.sub("\s+"," ", bill)
bill[:1000]

' Project Gutenberg s The Complete Works of William Shakespeare by William Shakespeare This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever You may copy it give it away or re use it under the terms of the Project Gutenberg License included with this eBook or online at www gutenberg org If you are not located in the United States you ll have to check the laws of the country where you are located before using this ebook Title The Complete Works of William Shakespeare Author William Shakespeare Release Date January EBook Last Updated November Language English Character set encoding UTF START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE The Complete Works of William Shakespeare by William Shakespeare Contents THE SONNETS ALL S WELL THAT ENDS WELL THE TRAGEDY OF ANTONY AND CLEOPATRA AS YOU LIKE IT THE COMEDY OF ERRORS THE TRAGEDY OF CORIOLANUS CYMBELINE THE TRAGE

In [36]:
# make a list of unique characters that appear in the file
chars = list(set(bill))

# dictionaries to convert letters to numbers, and take numbers to get back their letters.
char_indices = {c:i for i,c in enumerate(chars)}
indices_char = {i:c for i,c in enumerate(chars)}

In [6]:
bill_encoded = [char_indices[c] for c in bill]
bill_encoded[:10]

[28, 9, 15, 7, 44, 43, 20, 10, 28, 26]

In [13]:
# we now take the giant string and cut it up into more bite sized pieces
bill_seq = []
# for prediction later
next_char = []
# my seq_length is super short because any higher and I hit memrory errors
# when doing the onehot encoding, or my kernal dies later on
seq_length = 3

for i in range(0, len(bill_encoded) - seq_length, 5):
    bill_seq.append(bill_encoded[i: i + seq_length])
    next_char.append(bill_encoded[i + seq_length])

In [14]:
# homebrew onehot encoding

# create base all zero lists
X = np.zeros((len(bill_seq), seq_length, len(chars)))
y = np.zeros((len(bill_seq), len(chars)))

# activate the onehot
for i, seq in enumerate(bill_seq):
    for j, char in enumerate(seq):
        X[i, j, char] = 1
    
    y[i, next_char[i]] = 1

In [18]:
# build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [37]:
import sys

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(bill) - seq_length - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = bill[start_index: start_index + seq_length]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seq_length, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [38]:
import random

model.fit(X, y,
         batch_size=512,
         epochs=1,
         callbacks=[print_callback])

1006592/1007125 [============================>.] - ETA: 0s - loss: 1.6592
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " th"
 the the with the stand the prove the fath the so may the part and the shall the shall the come the shall have the shall the shall the s shall the shall the stand the come the me the will the s the so shall the s be man and the so see the with the shall the the shall and the pring and the so may the the man the see that hear the will the have the s so may the do the come the so see the shall and the 
----- diversity: 0.5
----- Generating with seed: " th"
 this turn d thou senge of his not the be shall of the have Her disportues not the will d his and the pers to king of at the me the pain they to be stron the shall that mad She your fath that the ruly I hardon ther that the dead the good lord that we and the in and for breat I am so many are with the more the shall the with the wearing him me of his a good to and that we h

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN